# Naiwny Bayes

Notatnik ten zawiera implementacją naiwnego klasyfikatowa Bayesa wykorzystując bibliotekę `numpy`. Działanie klasyfikatora jest porównane implementacja ``GaussianNB`` w bibliotece `sklearn`.

# Naive Bayes

This notebook contains a naive Bayesian implementation using the `numpy` library. Classifier operation is compared to the implementation of `` GaussianNB '' in the library `sklearn`.

Z twierdzenia Bayesa mamy:

$$P(J=j|X=x)=\frac{P(X=x|J=j)P(J=j)}{\sum_{j\in\{1,\ldots L\}}P(X=x|J=j)P(J=j)}$$

$$ p_j(x)=\frac{p_jf_j(x)}{f(x)} $$

Wykorzystamy te formuły w budowie klasyfikatora [Bayesa](http://books.icse.us.edu.pl/runestone/static/ai/KlasyfikacjaWOparciuPodobienstwoDoWzorca/OptymalnyKlasyfikatorStatystyczny.html).

From Bayes' theorem we have:

$$P(J=j|X=x)=\frac{P(X=x|J=j)P(J=j)}{\sum_{j\in\{1,\ldots L\}}P(X=x|J=j)P(J=j)}$$

$$ p_j(x)=\frac{p_jf_j(x)}{f(x)} $$

We will use these formulas in the construction of the [Bayesian] classifier.

In [ ]:
import numpy as np 
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn import datasets 
# dataset import 
iris = datasets.load_iris()

gnb = GaussianNB()

X = iris.data

y = iris.target
y = np.array(y)

train, test, train_targets, test_targets = \
   train_test_split(X, y, test_size=0.31)

# train 
clf = gnb.fit(train, train_targets)

# Test
Z = clf.predict(test)

In [ ]:
train.shape,test.shape

### Wyznacz unikalne klasy na zbiorze trenującym.

### Determine unique classes on the training set.

In [ ]:
Nlabels = None
### BEGIN SOLUTION
Nlabels = np.unique(train_targets)
Nlabels
### END SOLUTION

In [ ]:
np.testing.assert_array_equal(Nlabels,np.array([0, 1, 2]))

### Jaki jest procent poprawych odpowiedzi?

Implementacja ``GaussianNB`` daje tę odpowiedź w następujący sposób. Spróbuj otrzymać tą liczbę korzystając wyłącznie z funkcjonalności ``numpy`` i/lub języka Python.

### What is the percentage of correct answers?

The implementation of ``GaussianNB`` gives this answer as follows. Try to get this number using only the ``numpy`` functionality and / or Python language.

In [ ]:
clf.score(test,test_targets)

In [ ]:
correct = None
### BEGIN SOLUTION
correct = np.sum(Z==test_targets)/np.size(Z)
### END SOLUTION

In [ ]:
assert correct==clf.score(test,test_targets)

### Które odpowiedzi są złe:

### Which answers are wrong:

In [ ]:
bad_idx = np.where(Z!=test_targets)
bad_idx

### Parametry klasyfikatora

### Classifier parameters

In [ ]:
clf.theta_

In [ ]:
clf.sigma_

In [ ]:
ith = 2
np.mean(train[train_targets == ith],axis=0)

In [ ]:
np.var(train[train_targets == ith],axis=0)

## Implementacja naiwnego klasyfikatora Bayesa

**1\. Oblicz częstość występowania poszczególnych klasy $j$ w zbiorze treningowym $p_{j}$**

Jest to prawdopodobieństwo *a priori*.

## Implementation of the naive Bayes classifier

**1 \. Calculate the frequency of individual $j$ classes in the $p_{j}$** train set

This is the probability *a priori*.

In [ ]:
p = None
### BEGIN SOLUTION
p = [np.sum(train_targets==ith)/train_targets.shape[0] for ith in range(3) ]
### END SOLUTION

In [ ]:
np.testing.assert_allclose(p,clf.class_prior_)
### BEGIN HIDDEN TESTS
np.testing.assert_allclose( gnb.class_count_/np.sum(gnb.class_count_), gnb.class_prior_ )
### END HIDDEN TESTS

**2\. Oblicz wartość średnią dla każdej cechy z każdej klasy.**

Niech $\mu_{ij}$ oznacza  wartość średnią dla $j$-tej zmiennej w $i$-tej klasie, wtedy:
   
$$ \mu_{ij} =  \langle x_j \rangle_{ \forall x_j \in \;\mathrm{label} \;
i } $$

**2 \. Calculate the average value for each feature in each class.**

Let $\mu_{ij}$ be the average value for $j$ of this variable in $i$ of this class, then:
   
$$ \mu_{ij} =  \langle x_j \rangle_{ \forall x_j \in \;\mathrm{label} \;
i } $$

In [ ]:
mu = None
### BEGIN SOLUTION
mu = np.stack([np.mean(train[train_targets == ith],axis=0) for ith in [0,1,2]])
### END SOLUTION

In [ ]:
np.testing.assert_allclose(mu,clf.theta_)

In [ ]:
mu.shape

**3\. Oblicz wariancję $j$-tej zmiennej w $i$-tej klasie.**

   $$ \sigma_{ij}^2 = \mathrm{Var} [x_j] _ {\;\;{ \forall x_j \in \mathrm{class}\; i}} $$

**3 \. Calculate the variance of $j$ of this variable in $i$ of this class.**

   $$ \sigma_{ij}^2 = \mathrm{Var} [x_j] _ {\;\;{ \forall x_j \in \mathrm{class}\; i}} $$

In [ ]:
sigma2 = None
### BEGIN SOLUTION
sigma2 = np.stack([np.var(train[train_targets == ith],axis=0) for ith in [0,1,2]])
### END SOLUTION

In [ ]:
np.testing.assert_almost_equal(sigma2,clf.sigma_)

**4\. Oblicz prawdopodobieństwo *a posteriori* klasy $i$ dla danego wektora zmiennych  $\mathbf{x}$.**

Niech $k$ oznacza liczbę cech (zmiennych). W naszym przypadku mamy $k=4$.

Dla $i$-tej klasy mamy:


$$ P_i(\mathbf{x}) \simeq p_i f_i = p_i \frac{1}{\sqrt{(2\pi)^k\Pi_{j=1}^k\sigma_{ij}^2} } e^{ -\displaystyle\sum_{j=1}^{k}\frac{(x_j-\mu_{ij})^2}{2\sigma_{ij}^2} }
 $$
 
By otrzymać prawdopodobieństwa należy unormować $P_i$ dla każdego przypadku tak by suma $\sum_i P_i(\mathbf{x})=1$

**4 \. Calculate the * a posteriori * probability of the $i$ class for a given vector of $\mathbf{x}$.** variables

Let $k$ denote the number of features (variables). In our case, we have $k=4$.

For $i$ this class we have:


$$ P_i(\mathbf{x}) \simeq p_i f_i = p_i \frac{1}{\sqrt{(2\pi)^k\Pi_{j=1}^k\sigma_{ij}^2} } e^{ -\displaystyle\sum_{j=1}^{k}\frac{(x_j-\mu_{ij})^2}{2\sigma_{ij}^2} }
 $$
 
To obtain probabilities, $P_i$ should be normalized for each case so that the sum of $\sum_i P_i(\mathbf{x})=1$

In [ ]:
x  = test
P = None
k = test.shape[1] 

print ('number of features: k=',k)
### BEGIN SOLUTION
# single loop
P = np.stack( [p[ith]*1/np.sqrt( (2*np.pi)**k * np.prod(sigma2[ith])) * \
               np.exp( - 1/2.0* np.sum( (x-mu[ith])*(x-mu[ith])/sigma2[ith],axis=1 ) ) \
               for ith in range(3)]).T
# no loop
P = p/np.sqrt( (2*np.pi)**k * np.prod(sigma2,axis=1)) *\
  np.exp(-0.5*np.sum( (x[:,np.newaxis,:]-mu[np.newaxis,:,:])**2/sigma2[np.newaxis,:,:],axis=2) )

P = P/np.sum(P,axis=1)[:,np.newaxis]
### END SOLUTION

In [ ]:
sigma2.shape,x.shape,P.shape,np.unique(test_targets)

In [ ]:
np.testing.assert_almost_equal(P,clf.predict_proba(x))

**5\. Wyznacz klasę dla której prawdopodobieństwo *a posteriori* jest największe** 

 $i$ : $\quad\textrm{ gdy } P_i(x)=\max_{1\leq j\leq L} P_j(x)$


**5 \. Determine the class for which the * a posteriori * probability is the highest**

 $i$: $\quad\textrm{ gdy } P_i(x)=\max_{1\leq j\leq L} P_j(x)$

In [ ]:
prediction = None
### BEGIN SOLUTION
prediction = np.argmax(P,axis=1)
### END SOLUTION

In [ ]:
np.testing.assert_equal( prediction, clf.predict(test))

In [ ]:
print(clf.predict(test) )
print(prediction)

In [ ]:
clf.score(test,test_targets)